.pio file also contains a helper function to set up a PIO state machine for correct execution of this program:

In [9]:
#float div 计算

#float div = clock_get_hz(clk_sys) / (freq * cycles_per_bit);
# sm_config_set_clkdiv(&c, div);

clk_sys = 150* 10**6 #Hz 150M

freq = 800 * 10**3 #Hz   800K

cycles_per_bit = 18

div =   clk_sys/(freq * cycles_per_bit)

#div = f_sys/ f_2

#f_2 = pio指令频率* 每bit所需指令周期数量   # 需要指令时间越多，所需频率f_2 就越大

print(div)

10.416666666666666


In [18]:
clk_sys = 150* 10**6 #Hz 150M
max_clcles = 13

target_frequency = clk_sys/ max_clcles
print(f"信号最大频率: {target_frequency/10**6} Mhz")


print(f"T: {1/target_frequency*10**6} us  = {1/target_frequency*10**9} ns")

print(f"计算估计时间 {5}us")
print(f"控制频率 f{1/5  *1000} khz")

信号最大频率: 11.538461538461538 Mhz
T: 0.08666666666666667 us  = 86.66666666666667 ns
计算估计时间 5us
控制频率 f200.0 khz


In [ ]:
static inline void hello_program_init(PIO pio, uint sm, uint offset, uint pin) {

pio_sm_config c = hello_program_get_default_config(offset);
// Map the state machine's OUT pin group to one pin, namely the `pin`
// parameter to this function.
sm_config_set_out_pins(&c, pin, 1);
// Set this pin's GPIO function (connect PIO to the pad)
pio_gpio_init(pio, pin);

// Set the pin direction to output at the PIO
pio_sm_set_consecutive_pindirs(pio, sm, pin, 1, true);

// Load our configuration, and jump to the start of the program
pio_sm_init(pio, sm, offset, &c);

// Set the state machine running
pio_sm_set_enabled(pio, sm, true);







}

Each PIO instruction is 16 bits in size.



CMakeList.txt

need 
pico_generate_pio_header(test ${CMAKE_CURRENT_LIST_DIR}/piotest.pio)

c 
need
#include "piotest.pio.h" // generated by cmake


pio 辅助函数内设置 GPIO目标

考虑以下3点

1. The state machine needs to be told *which GPIO* or GPIOs to output to. 
There are four different *pin groups* which
are used by different instructions in different situations;
 here we are using the *out pin group*, because we are just
using an out instruction.

2. The GPIO also needs to be told that *PIO is in control of it* (GPIO function select)

3. If we are using the pin for output only, we need to make sure that PIO is driving *the output enable line* high. PIO can
drive this line up and down programmatically using e.g. an out pindirs instruction, but here we are setting it up
before starting the program.

### 3.2.1.2. C Program

PIO won’t do anything until it’s been configured properly, so we need some software to do that.

The PIO file we just
looked at — hello.pio — is converted automatically (we will see later how) into a header containing our assembled PIO
program binary, any helper functions we included in the file, and some useful information about the program. We
include this as *hello.pio.h*.

sm  = state machine

Each PIO instruction is 16 bits in size. Generally, 5 of those bits in each instruction are used for the “delay” which is
usually 0 to 31 cycles (after the instruction completes and before moving to the next instruction). If you have read the
PIO chapter of the RP2350 Datasheet, you may have already know that these **5 bits can be used for a different purpose:**

delay 5bit 起其他作用

.side_set 1

This directive .side_set 1 says we’re stealing one of those delay bits to use for **"side-set"**.

The state machine will use this
bit to **drive the values of some pins**, once per instruction, in addition to what the instructions are themselves doing.

This
is very useful for**high frequency use** cases (e.g. pixel clocks for DPI panels), but also for **shrinking program size**, to fit
into the shared instruction memory.

Note that *stealing one bit has left our delay range from 0-15 (4 bits)*, but that is quite natural because you rarely want to
mix side-set with lower frequency stuff.

we have to specify a side-set value for
**every instruction** in the program.

This is the **side N** you will see on each instruction in the listing.

.define lets you declare constants. The public keyword means that the assembler will also write out the value of the define in the output file for use by other software:

In [ ]:
.define public T1 2
.define public T2 5
.define public T3 3

in the context of the SDK, this is a **#define**. We are going to use T1, T2
and T3 in calculating the delay cycles on each instruction.

In [ ]:
bitloop:

This is a label. A label tells the assembler that this point in your code is interesting to you, and you want to refer to it
later by name. Labels are mainly used with jmp instructions.

out x, 1 side 0 [T3 - 1] ; Side-set still takes place when instruction stalls

In [ ]:
out x, 1 side 0 [T3 - 1] ;

This is an out instruction.

out takes some bits from the output shift register (OSR), and writes them somewhere
else. In this case, the OSR will contain pixel data destined for our LEDs.


out pindirs 指令可更改pio pin 的方向（输入  输出）

Data request signals

The DMA can transfer data incredibly fast, and almost invariably this will be much faster than your PIO
program actually needs. The DMA paces itself based on a data request handshake with the state
machine, so there’s no worry about it overflowing or underflowing a FIFO, as long as you have selected
the correct **DREQ signal**. The state machine coordinates with the DMA to tell it when it has room
available in its TX FIFO, or *data available in its RX FIFO*.